<a href="https://colab.research.google.com/github/BruceAuHK/Dashboard_FFO/blob/master/Plotly_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import urllib.request
import json
import ssl
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

context = ssl._create_unverified_context()

def get_data_from_HKMA(url):
    # Balance sheet
    url = url

    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        with urllib.request.urlopen(url+"&offset="+str(offset), context=context) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['datasize'] <= 0:
            break
        else:
            offset += 100

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records


print("dependencies ready")


dependencies ready


# **Monetary Base**

A part of the monetary liabilities of a central bank. The Monetary Base is defined, at the minimum, as the sum of the currency in circulation (banknotes and coins) and the balance of the banking system held with the central bank (the reserve balance or the clearing balance). In Hong Kong, the Monetary Base comprises Certificates of Indebtedness (for backing the banknotes issued by the note-issuing banks), government-issued currency in circulation, the balance of the clearing accounts of banks kept with the HKMA (the Aggregate Balance), and Exchange Fund Bills and Notes.


In [ ]:
MBC = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/monetary-operation/monetary-base-endperiod?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. Size of the Monetary Base was published since 25 November 1998.
mbc_df = pd.json_normalize(get_data_from_HKMA(MBC))

#Convert dataframe to dataframe long format
mbc_dfl = pd.melt(mbc_df, id_vars = ['end_of_month'], value_vars =['cert_of_indebt', 'gov_notes_coins_circulation' , 'aggr_balance_bf_disc_win' , 'outstanding_efbn'],var_name='Size of the Monetary Base', value_name='HK$ million' )

fig = px.line(mbc_dfl, x='end_of_month' , y='HK$ million', color = 'Size of the Monetary Base')
fig.update_layout(title_text="Monetary Base – End of period figures")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()


#1. Money Multiplier = M3 / MB (HKD)

The total stock of money available in the economy. Hong Kong has three measures of money supply:

Money Supply definition 1 (M1): The sum of legal tender notes and coins held by the public plus customers' demand deposits placed with banks.

Money Supply definition 2 (M2): M1 plus customers' savings and time deposits with banks plus negotiable certificates of deposit (NCDs) issued by banks held outside the banking sector.

Money Supply definition 3 (M3): M2 plus customers' deposits with restricted licence banks and deposit-taking companies plus NCDs issued by these institutions held outside the banking sector.

Among these three series, HK M1 exhibits a significant seasonal pattern, whereas there is no strong evidence of seasonality in broad money (HK M2 and HK M3). Seasonally adjusted series of HK M1 and its components (i.e. cash held by the public and demand deposits) are compiled and published by the HKMA.

In [ ]:
# ---------------- 1. Money Multiplier = M3 / MB (HKD) ----------------

MSA = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/money/supply-adjusted?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. There is a break in data series due to the inclusion of short-term Exchange Fund placements of less than one month since April 1997.
msa_df = pd.json_normalize(get_data_from_HKMA(MSA))

m3_hkd = msa_df[['end_of_month','m3_hkd']]

mb_tot = mbc_df[['end_of_month','mb_bf_disc_win_total']]

m3_hkd_Over_mb_tot = pd.merge(m3_hkd, mb_tot, on='end_of_month')

m3_hkd_Over_mb_tot['m3Overmb'] = (m3_hkd_Over_mb_tot['m3_hkd'] / m3_hkd_Over_mb_tot['mb_bf_disc_win_total'])


fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3Overmb", title='M3(hkd) / MB(Total)')

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")
fig.update_yaxes(title_text="M3 / MB")
fig.show()




#2. Total deposits & M3 (HKD only)

In [ ]:
TD = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/customer-deposits-by-currency?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

td_records_df = pd.json_normalize(get_data_from_HKMA(TD))

deposits_all = td_records_df[['end_of_month','deposits_all']]

m3_hkd_and_deposits_all  = pd.merge(m3_hkd, deposits_all, on='end_of_month')

#Convert dataframe to dataframe long format
m3_hkd_and_deposits_all_dfl = pd.melt(m3_hkd_and_deposits_all, id_vars = ['end_of_month'], value_vars =['m3_hkd', 'deposits_all'],var_name='var', value_name='HK$ million' )

fig = px.line(m3_hkd_and_deposits_all_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.update_layout(title_text="Total deposits & M3 (HKD)")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

#3. Loan to Deposits


## Loan to Deposits

In [141]:
BS = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/balance-sheet-ais?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

bs_records_df = pd.json_normalize(get_data_from_HKMA(BS))

fig = make_subplots(rows=1, cols=3)

#Assets Loans and advances to customers HK$ - asst_loanadvtocust_hkd
#Liabilities Deposits from customers HK$  - asst_loanadvtocust_hkd
#Assets Loans and advances to customers F.C. - asst_loanadvtocust_fc
#Liabilities Deposits from customers F.C.  - liab_depcust_fc
#Assets Loans and advances to customers Total - asst_loanadvtocust_total
#Liabilities Deposits from customers Total  - liab_depcust_total

bs_records_df['LtoD_hkd'] = (bs_records_df['asst_loanadvtocust_hkd'] / bs_records_df['liab_depcust_hkd'])
bs_records_df['LtoD_fc'] = (bs_records_df['asst_loanadvtocust_fc'] / bs_records_df['liab_depcust_fc'])
bs_records_df['LtoD_total'] = (bs_records_df['asst_loanadvtocust_total'] / bs_records_df['liab_depcust_total'])

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_hkd"], mode='lines',
              name ='Loan to Deposits(hkd only)'), row = 1, col=1)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_fc"], mode='lines',
              name ='Loan to Deposits(fc only)'), row = 1, col=2)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["LtoD_total"], mode='lines',
              name ='Loan to Deposits (Total))'), row = 1, col=3)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")
fig.update_yaxes(title_text="Loan to Deposits")
fig.update_layout(title_text="Loan to Deposits")

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

In [142]:
fig = make_subplots(rows=1, cols=3)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["asst_loanadvtocust_hkd"], mode='lines',
              name ='Loan(hkd only)'), row = 1, col=1)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["liab_depcust_hkd"], mode='lines',
              name ='Deposits(hkd only)'), row = 1, col=1)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["asst_loanadvtocust_fc"], mode='lines',
              name ='Loan(fc only)'), row = 1, col=2)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["liab_depcust_fc"], mode='lines',
              name ='Deposits(fc only)'), row = 1, col=2)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["asst_loanadvtocust_total"], mode='lines',
              name ='Loan(Total))'), row = 1, col=3)

fig.add_trace(go.Scatter(x=bs_records_df["end_of_month"], y =bs_records_df["liab_depcust_total"], mode='lines',
              name ='Deposits (Total))'), row = 1, col=3)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")
fig.update_yaxes(title_text="HK$ million")
fig.update_layout(title_text="Loan and Deposits")




fig.show()

#4.Centaline index

Announced every month, latest on 2020/ 07/ 18, reflecting secondary private residential property price for 2020/05 (based on formal sale & purchase date; on average, formal S&P are signed within 14 days after preliminary S&P)

In [143]:
!wget https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/cci.csv
cci = pd.read_csv('cci.csv')
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=cci["date"], y = cci["cci"], mode='lines', name="Centaline index"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   

# Add figure title
fig.update_layout( xaxis_range=['1997-7','2020-06'],
    title_text="CCI VS MB"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>CCI</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))


fig.show()


--2020-07-21 02:06:03--  https://raw.githubusercontent.com/BruceAuHK/Dashboard_FFO/master/cci.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4205 (4.1K) [text/plain]
Saving to: ‘cci.csv.10’

cci.csv.10          100%[===================>]   4.11K  --.-KB/s    in 0s      

2020-07-21 02:06:03 (44.5 MB/s) - ‘cci.csv.10’ saved [4205/4205]



#5.MB / AB vs HIBOR

## MB VS HIBOR

In [144]:
HIBOR = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-endperiod?sortby=end_of_month&sortorder=asc'
HIBOR_df = pd.json_normalize(get_data_from_HKMA(HIBOR))

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['mb_bf_disc_win_total'], mode='lines', name="MB"),secondary_y=True,)   

# Add figure title
fig.update_layout( xaxis_range=['1998-7','2020-06'],
    title_text="MB VS HIBOR"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")


# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)


# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

 


## AB VS HIBOR

### Aggregate Balance

The sum of balances in the clearing accounts and reserve accounts maintained by commercial banks with the central bank. In Hong Kong, this refers to the sum of the balances in the clearing accounts maintained by the banks with the HKMA for settling interbank payments and payments between banks and the HKMA. The Aggregate Balance is a part of the Monetary Base.

In [145]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=HIBOR_df["end_of_month"], y = HIBOR_df["ir_1m"], mode='lines', name="HIBOR"),secondary_y=False)   

fig.add_trace(go.Scatter(x=mbc_df['end_of_month'], y = mbc_df['aggr_balance_bf_disc_win'], mode='lines', name="AB"),secondary_y=True,)   

# Add figure title
fig.update_layout( xaxis_range=['1998-7','2020-06'],
    title_text="AB VS HIBOR"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>percent per annum</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>HK$ million</b>", secondary_y=True)


# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

 


#6. FPS

## Transactions and Registrations

In [146]:
fps_val_hkd = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-turnover-fps-hkd-payment-val?sortby=end_of_month&sortorder=asc'
fps_val_hkd_df = pd.json_normalize(get_data_from_HKMA(fps_val_hkd))

fps_reg = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/ch-statistics-fps-reg?sortby=end_of_month&sortorder=asc'
fps_reg_df = pd.json_normalize(get_data_from_HKMA(fps_reg))

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(x=fps_val_hkd_df['end_of_month'], y = fps_val_hkd_df['total'], name="FPS Turnover of HKD FPS payment (Value in HK$ thousand)"),secondary_y=False)   

fig.add_trace(go.Scatter(x=fps_reg_df['end_of_month'], y = fps_reg_df['total_grand_total'], mode='lines+markers', name="FPS Addressing Service Registrations Total Grand-total"),secondary_y=True,)  

# Add figure title
fig.update_layout( xaxis_range=['2018-9','2020-06'],
    title_text="Transactions and Registrations"
)

# Set x-axis title
fig.update_xaxes(title_text="end_of_month")

# Set y-axes titles
fig.update_yaxes(title_text="<b>FPS Turnover of HKD FPS payment (Value in HK$ thousand)</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>FPS Addressing Service Registrations Total Grand-total</b>", secondary_y=True)

# Adjust legend anchor
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

 
